# Прогнозирование с помощью регрессии

Класс моделей ARIMA недостаточно богат для наших данных: с их помощью, например, никак нельзя учесть взаимосвязи между рядами. Это можно сделать с помощью векторной авторегрессии VARIMA, но её питоновская реализация не позволяет использовать регрессионные признаки. Кроме того, авторегрессионный подход не позволяет учитывать, например, взаимодействия между сезонными компонентами. Вы могли заметить, что форма суточных сезонных профилей в будни и выходные немного разная; явно моделировать этот эффект с помощью ARIMA не получится.

Нам нужна более сложная модель. Давайте займёмся сведением задачи массового прогнозирования рядов к регрессионной постановке!

Вам понадобится много признаков. Некоторые из них у вас уже есть — это:

- идентификатор географической зоны
- дата и время
- количество поездок в периоды, предшествующие прогнозируемому
- синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA

Кроме того, не спешите выбрасывать построенный вами на прошлой неделе прогнозы — из них может получиться хороший признак для регрессии!

Вы можете попробовать разные регрессионный модели, но хорошие результаты, скорее всего, дадут такие, которые будут позволять признакам взаимодействовать друг с другом.

Поскольку прогноз нужен на 6 часов вперёд, проще всего будет построить 6 независимых регрессионных моделей — одна для прогнозирования $ \widehat{y}_{T+1|T} $, другая для $ \widehat{y}_{T+2|T} $ и т.д.

## Задание № 1

Для каждой из шести задач прогнозирования $ \widehat{y}_{T+i|T} ,i=1,…,6 $ сформируйте выборки. Откликом будет $ y_{T+i} $ при всевозможных значениях $ T $, а признаки можно использовать следующие:
- идентификатор географической зоны — категориальный
- год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так
- синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
- сами значения прогнозов ARIMA $ \widehat{y}^{ARIMA}_{T+i|T} $
- количество поездок из рассматриваемого района в моменты времени $ y_T,y_{T−1},…,y_{T−K} $(параметр $K$ можно подбирать; попробуйте начать, например, с 6)
- количество поездок из рассматриваемого района в моменты времени $ y_{T−24} ,y_{T−48},…,y_{T−24∗K_d} $ (параметр $K_d$ можно подбирать; попробуйте начать, например, с 2)
- суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц

Будьте внимательны при создании признаков — все факторы должны быть рассчитаны без использования информации из будущего: при прогнозировании $ \widehat{y}_{T+i|T} ,i=1,…,6 $ вы можете учитывать только значения $ y $ до момента времени $ T $ включительно.

In [1]:
import numpy as np
import pandas as pd
from glob import glob

import statsmodels.formula.api as smf
import statsmodels.api as sm

/Users/Ar4eBaT/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Перед созданием признаков необходимо загрузить данные, и совместить их между собой

In [383]:
file_path = glob("u_data/*.csv", recursive=True)
file_path = np.sort(file_path)[:-12]

In [384]:
def open_data(path):
    data_list = []
    for p in path:
        data = pd.read_csv(p, sep=",", index_col="time", parse_dates=["time"])
        data_list.append(data)
    return pd.concat(data_list)

In [385]:
data_orig = open_data(file_path)

In [386]:
data_means = data_orig.loc['2016.05.01 00:00:00':'2016.05.31 23:00:00'].groupby("region").mean()
regions = data_means[data_means.trips >= 5].index.values
del data_means

In [387]:
regions_check = data_orig.region.isin(regions)
data = data_orig[regions_check].loc['2013.01.01 00:00:00':'2016.06.30 23:00:00']

In [388]:
temp = [data[data.region == r].trips for r in regions]
data = pd.concat(temp, axis=1)
data.columns = ["region_" + str(region) for region in regions]
del temp
data.head(5)

,region_1075,region_1076,region_1077,region_1125,region_1126,region_1127,region_1128,region_1129,region_1130,region_1131,...,region_1630,region_1684,region_1733,region_1734,region_1783,region_2068,region_2069,region_2118,region_2119,region_2168
time,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,75.0,149.0,88.0,95.0,297.0,538.0,594.0,769.0,952.0,267.0,...,12.0,0.0,9.0,113.0,27.0,19.0,6.0,68.0,18.0,67.0
2013-01-01 01:00:00,108.0,204.0,81.0,126.0,402.0,572.0,518.0,623.0,655.0,220.0,...,22.0,0.0,3.0,7.0,5.0,11.0,3.0,82.0,1.0,15.0
2013-01-01 02:00:00,82.0,162.0,83.0,135.0,361.0,471.0,470.0,495.0,484.0,156.0,...,23.0,0.0,1.0,6.0,3.0,2.0,0.0,28.0,0.0,16.0
2013-01-01 03:00:00,79.0,122.0,24.0,106.0,289.0,482.0,448.0,467.0,337.0,102.0,...,25.0,0.0,1.0,1.0,7.0,2.0,0.0,6.0,0.0,1.0
2013-01-01 04:00:00,35.0,89.0,20.0,59.0,181.0,332.0,377.0,343.0,354.0,92.0,...,26.0,0.0,2.0,5.0,2.0,3.0,0.0,30.0,2.0,16.0


Создаем признаковое пространство из значений года, месяца, дня и часа, разделяя значения дат и времени у исходной выборки.

In [ ]:
cluster_0 = (['region_1175', 'region_1337', 'region_1338', 'region_1339', 'region_1386', 'region_1442'], (6, 12))
cluster_1 = (['region_1129', 'region_1130', 'region_1176', 'region_1177', 'region_1178', 'region_1227',
             'region_1228', 'region_1278'], (4, 14))
cluster_2 = (['region_1127', 'region_1128', 'region_1131', 'region_1179', 'region_1182', 'region_1224',
             'region_1229', 'region_1230'], (6, 10))
cluster_3 = (['region_1076', 'region_1077', 'region_1132', 'region_1181', 'region_1183', 'region_1184',
             'region_1234', 'region_1235', 'region_1279', 'region_1280', 'region_1284', 'region_1285',
             'region_1286', 'region_1287', 'region_1331', 'region_1332', 'region_1333', 'region_1334',
             'region_1335', 'region_1336', 'region_1383', 'region_1384', 'region_1385', 'region_1434'], (5, 12))
cluster_4 = (['region_1480', 'region_1483', 'region_1530', 'region_1580'], (6, 9))
cluster_5 = (['region_1075', 'region_1125', 'region_1126', 'region_1180', 'region_1231', 'region_1232',
             'region_1233', 'region_1281', 'region_1282', 'region_1283', 'region_1382'], (6, 13))
cluster_6 = (['region_1172', 'region_1173', 'region_1174', 'region_1225'], (6, 15))
cluster_7 = (['region_1326', 'region_1327', 'region_1376', 'region_1377', 'region_1378', 'region_1426'], (6, 5))
cluster_8 = (['region_2168'], (4, 10))
cluster_9 = (['region_1221', 'region_1222', 'region_1223', 'region_1272', 'region_1273', 'region_1274',
             'region_1380'], (6, 4))
cluster_10 = (['region_1389', 'region_1390', 'region_1439'], (4, 7))
cluster_11 = (['region_2068', 'region_2069', 'region_2118', 'region_2119'], (6, 5))
cluster_12 = (['region_1684', 'region_1733', 'region_1734', 'region_1783'], (6, 4))
cluster_13 = (['region_1431'], (6, 3))
cluster_14 = (['region_1387', 'region_1388', 'region_1435', 'region_1436', 'region_1437', 'region_1438'], (3, 7))
cluster_15 = (['region_1630'], (6, 3))
cluster_16 = (['region_1482', 'region_1532', 'region_1533'], (6, 1))
cluster_17 = (['region_1441'], (5, 3))

clusters = [cluster_0, cluster_1, cluster_2, cluster_3, cluster_4, cluster_5, cluster_6, cluster_7, cluster_8,
           cluster_9, cluster_10, cluster_11, cluster_12, cluster_13, cluster_14, cluster_15, cluster_16,
            cluster_17]

In [ ]:
# Функция построения регрессионного признака по формулам синуса и косинуса

def season_attribute(func_name, start_T, T, season_length, season_name, K):
    if func_name not in ["sin", "cos"]:
        raise ValueError("Set func_name the value is sin or cos")
    
    func = np.sin if func_name is "sin" else np.cos
    
    if start_T < 1:
        raise ValueError("Set start_T equal or greater then 1")
        
    t_interval = np.arange(start_T, T + 1, dtype=int)
    values_dict = dict()
    for k in range(1, K + 1):
        key = func_name + "_" + season_name + "_" + str(k)
        values_dict[key] = func(t_interval * 2 * np.pi * k / season_length)
    return pd.DataFrame(values_dict)

In [459]:
# Функция создания признака тренда

def trend_attribute(start_T, T):
    if start_T < 1:
        raise ValueError("Set start_T equal or greater then 1")  
    t_interval = np.arange(start_T, T + 1, dtype=int)
    return pd.DataFrame(t_interval, columns=["Trend"])

In [460]:
# Функция построения значений ряда Фурье по значениям k для недели и года.

def set_attributes(cluster, week_k=0, year_k=0, trend=True, start=1, end=1):
    data_list = []
    
    if cluster is not None:
        data_list.extend([cluster])
    
    if end <= 1:
        end = cluster.shape[0]
    
    day_hours = 24

    # Регрессионные признаки по недельной сезонности
    week_hours = day_hours * 7
    if week_k > 0:
        sin_week_data = season_attribute("sin", start, end, week_hours, "week", week_k).set_index(cluster.index)
        cos_week_data = season_attribute("cos", start, end, week_hours, "week", week_k).set_index(cluster.index)
        data_list.extend([sin_week_data, cos_week_data])

    # Регрессионные признаки по годовой сезонности
    year_hours = 8766
    if year_k > 0:
        sin_year_data = season_attribute("sin", start, end, year_hours, "year", year_k).set_index(cluster.index)
        cos_year_data = season_attribute("cos", start, end, year_hours, "year", year_k).set_index(cluster.index)
        data_list.extend([sin_year_data, cos_year_data])
    
    if trend is True:
        trend_data = trend_attribute(start, end).set_index(cluster.index)
        data_list.extend([trend_data])

    cluster = pd.concat(data_list, axis=1)
        
    return cluster.iloc[:,1:]

In [ ]:
# Получение параметров региона для построения признакового пространства из значений ряда Фурье, в зависимости от
# принадлежности региона к какому-либо из кластеров

def get_params(region):
    for cluster in clusters:
        if region in cluster[0]:
            return cluster[1]

In [ ]:
# Функция получения значений поездок за предыдущие k-часов

def trips_short(region, k):
    trips = np.zeros((data.shape[0]-k-1, k+1))
    for i in range(k+1, data.shape[0]):
        trips[i-k-1] = data[region].iloc[i-k-1:i][::-1].get_values()
    
    columns = ["trips_short_t"]
    columns.extend(list(map(lambda num: "trips_short_t_" + str(num), range(1, k+1))))
    
    return pd.DataFrame(trips, columns=columns)

In [ ]:
# Функция получения значение поездок за предыдущие k-дней

def trips_days(region, k):
    trips = np.zeros((data.shape[0]-24*k, k))
    for i in range(24*k, data.shape[0]):
        trips[i-24*k] = list(map(lambda num: data[region].iloc[i - 24*(num+1)], range(k)))
    
    columns = []
    columns.extend(list(map(lambda num: "trips_days_t_" + str(24*num), range(1, k+1))))
    
    return pd.DataFrame(trips, columns=columns)

In [ ]:
# Функция получения суммы поездок за указанный период

def count_trips(region, period):
    ct_hd = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        if i is 0:
            ct_hd[i] = data[region].iloc[0]
        elif i < period:
            ct_hd[i] = data[region].iloc[:i+1].sum()
        else:
            ct_hd[i] = data[region].iloc[i-period:i+1].sum()
    return ct_hd

In [ ]:
# Функция построения признаковых пространств по суммам за период половины суток, суток, недели и месяца

def all_count_trips(region):
    count_trips_params = [("count_trips_semiday", 12), ("count_trips_allday", 24), ("count_trips_week", 24*7),
                         ("count_trips_month", 24*30)]
    ct_list = []
    columns = []
    for params in count_trips_params:
        ct_list.append(count_trips(region, params[1]))
        columns.append(params[0])
    
    ct_list = np.array(ct_list).transpose()
    return pd.DataFrame(ct_list, columns=columns)

In [ ]:
# Функция, объединяющая вышеопределенные, для построения признакового пространства заданного региона

def create_attributes(sin_cos, k_sh, k_days):
    region = sin_cos.region[0]

    date_attrib = []
    for date in data.index:
        date_attrib.append([date.year, date.strftime("%B"), date.day, date.weekday_name, date.hour])
    date_attrib = pd.DataFrame(date_attrib, columns=["year", "month", "day", "dayofweek", "hour"])
    t_sh = trips_short(region, k_sh)
    t_days = trips_days(region, k_days)
    c_t = all_count_trips(region)

    min_val_count = np.min([date_attrib.shape[0], t_sh.shape[0], t_days.shape[0], c_t.shape[0]])
    
    date_attrib = date_attrib.iloc[-min_val_count:]
    date_attrib.index = range(min_val_count)
    
    t_sh = t_sh.iloc[-min_val_count:]
    t_sh.index = range(min_val_count)
    
    t_days = t_days.iloc[-min_val_count:]
    t_days.index = range(min_val_count)
    
    c_t = c_t.iloc[-min_val_count:]
    c_t.index = range(min_val_count)
    
    sin_cos = sin_cos.iloc[-min_val_count:]
    sin_cos.index = range(min_val_count)
    
    res_data = [date_attrib, sin_cos, t_sh, t_days, c_t]
    all_data = pd.concat(res_data, axis=1)
    
    data_list = []
    for k in range(1, 7):
        target = data[region].iloc[-min_val_count+k:]
        target = pd.DataFrame(target.get_values(), index=range(len(target)), columns=["trips"])
        data_list.append(pd.concat([target, all_data[:-k]], axis=1))
    return (data_list[0], data_list[1], data_list[2], data_list[3], data_list[4], data_list[5])

In [ ]:
# Функция выполняющая подготовку признаков, и применяет последовательное ранжирования для данных каждой из задач

def prepare_data(k_sh, k_days):
    attr_sin_cos = []
    for region in data.columns:
        params = get_params(region)
        attr = set_attributes(data[region], params[0], params[1])
        region_df = pd.DataFrame(np.repeat(region, data.shape[0]), index=attr.index, columns=["region"], dtype="category")
        attr_sin_cos.append(pd.concat([region_df, attr], axis=1))
    
    first_data = []
    second_data = []
    third_data = []
    fourth_data = []
    fifth_data = []
    sixth_data = []
    for sin_cos in attr_sin_cos:
        data_list = create_attributes(sin_cos, k_sh, k_days)
        first_data.append(data_list[0])
        second_data.append(data_list[1])
        third_data.append(data_list[2])
        fourth_data.append(data_list[3])
        fifth_data.append(data_list[4])
        sixth_data.append(data_list[5])
    
    first_data = pd.concat(first_data).fillna(0.0).sort_values(["year", "month", "day", "hour"])
    second_data = pd.concat(second_data).fillna(0.0).sort_values(["year", "month", "day", "hour"])
    third_data = pd.concat(third_data).fillna(0.0).sort_values(["year", "month", "day", "hour"])
    fourth_data = pd.concat(fourth_data).fillna(0.0).sort_values(["year", "month", "day", "hour"])
    fifth_data = pd.concat(fifth_data).fillna(0.0).sort_values(["year", "month", "day", "hour"])
    sixth_data = pd.concat(sixth_data).fillna(0.0).sort_values(["year", "month", "day", "hour"])
    
    return (first_data, second_data, third_data, fourth_data, fifth_data, sixth_data)

Для достаточной полноты данных было принято решение указать значение **k-часов** равным 24, а **k-суток** равным двум неделям.

Далее осуществляется подготовка данных, и сохранение их на компьютере.

In [ ]:
%%time
first_data, second_data, third_data, fourth_data, fifth_data, sixth_data = prepare_data(24, 14)

In [ ]:
first_data.to_csv("new_data/1. First_data.csv", index=False)

In [ ]:
second_data.to_csv("new_data/2. Second_data.csv", index=False)

In [ ]:
third_data.to_csv("new_data/3. Third_data.csv", index=False)

In [ ]:
fourth_data.to_csv("new_data/4. Fourth_data.csv", index=False)

In [ ]:
fifth_data.to_csv("new_data/5. Fifth_data.csv", index=False)

In [ ]:
sixth_data.to_csv("new_data/6. Sixth_data.csv", index=False)

## Задание № 2

Разбейте каждую из шести выборок на три части:

- обучающая, на которой будут настраиваться параметры моделей — всё до апреля 2016
- тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
- итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016

In [461]:
# Функция разделения единого массива данных на обучающую, тестовую и контрольную выборки

def split_data(input_data):
    train_data = pd.concat([input_data[input_data.year < 2016],
                            input_data[(input_data.year == 2016) & (input_data.month.astype(int) < 5)]])

    test_data = input_data[(input_data.year == 2016) & (input_data.month == "5")]
    control_data = pd.concat([test_data[-102:],
                             input_data[(input_data.year == 2016) & (input_data.month == "6")]])
    test_data = test_data[:-102]
    
    return train_data, test_data, control_data

На данном этапе происходит разделение данных из каждой задачи на три выборки, и сохраняет полученный кортеж в общем массиве.

In [119]:
file_path = np.sort(glob("new_data/*.csv", recursive=True))
input_data = []
for path in file_path:
    new_data = pd.read_csv(path)
    new_data.month = new_data.month.astype(str)
    input_data.append(new_data)
    
result_data = []
for i in range(len(input_data)):
    i_data = input_data[i]
    result_data.append(split_data(i_data))

## Задание № 3

Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016. Желательно, чтобы модель:

- допускала попарные взаимодействия между признаками
- была устойчивой к избыточному количеству признаков (например, использовала регуляризаторы)

Создание массива названий столбцов признаков, за исключением признаков значений года и дня, по причине их незначительного влияния на конечную модель.

In [24]:
attribute_cols = input_data[0].columns[(input_data[0].columns != "trips") & (input_data[0].columns != "year") & (input_data[0].columns != "day")]

'trips ~ Trend + cos_week_1 + cos_week_2 + cos_week_3 + cos_week_4 + cos_week_5 + cos_week_6 + cos_year_1 + cos_year_10 + cos_year_11 + cos_year_12 + cos_year_13 + cos_year_14 + cos_year_15 + cos_year_2 + cos_year_3 + cos_year_4 + cos_year_5 + cos_year_6 + cos_year_7 + cos_year_8 + cos_year_9 + count_trips_allday + count_trips_month + count_trips_semiday + count_trips_week + dayofweek + hour + month + region + sin_week_1 + sin_week_2 + sin_week_3 + sin_week_4 + sin_week_5 + sin_week_6 + sin_year_1 + sin_year_10 + sin_year_11 + sin_year_12 + sin_year_13 + sin_year_14 + sin_year_15 + sin_year_2 + sin_year_3 + sin_year_4 + sin_year_5 + sin_year_6 + sin_year_7 + sin_year_8 + sin_year_9 + trips_days_t_120 + trips_days_t_144 + trips_days_t_168 + trips_days_t_192 + trips_days_t_216 + trips_days_t_24 + trips_days_t_240 + trips_days_t_264 + trips_days_t_288 + trips_days_t_312 + trips_days_t_336 + trips_days_t_48 + trips_days_t_72 + trips_days_t_96 + trips_short_t + trips_short_t_1 + trips_short

В качестве модели, осуществляющей прогноз поездок по районам, был выбран **Случайный лес**, так как он позволяет признакам тесно взаимодейсвовать между собой.

Перед обучением модели, следует к категориальным признакам применить **бинарное кодирование**.

In [362]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer as DV

In [363]:
cat_attributes = ["region", "month", "dayofweek"]
regions = pd.unique(result_data[0][0]["region"])
month = list(map(lambda x: "m_" + str(x), (pd.unique(result_data[0][0]["month"]))))
dayofweek = pd.unique(result_data[0][0]["dayofweek"])
cat_columns = np.append(np.append(regions, month), dayofweek)

In [364]:
encoder = DV(sparse = False)
encoder.fit(result_data[0][0][cat_attributes].T.to_dict().values())
cat_att = pd.DataFrame(encoded_data, columns=cat_columns)

In [365]:
a = list(attribute_cols)
a.remove("month")
a.remove('dayofweek')
a.remove('hour')
a.remove('region')

В ходе обучения модели была сразу проведена предварительная оценка качества результата обучения алгорима на тестовой выборке.

In [443]:
%%time

results = []
for data in result_data:
    estimator = RandomForestRegressor()

    encoded_data = encoder.transform(data[0][cat_attributes].T.to_dict().values())
    cat_att = pd.DataFrame(encoded_data, columns=cat_columns)
    cat_att.index = data[0][a].index

    X_train = pd.concat([data[0][a], cat_att], axis=1)
    y_train = data[0]["trips"]

    fitted = estimator.fit(X_train, y_train)

    encoded_data = encoder.transform(data[1][cat_attributes].T.to_dict().values())
    cat_att = pd.DataFrame(encoded_data, columns=cat_columns)
    cat_att.index = data[1][a].index

    X_test = pd.concat([data[1][a], cat_att], axis=1)
    y_test = data[1]["trips"]

    prediction = fitted.predict(X_test)
    estimate = sum(np.abs(prediction.astype(int) - y_test)) / (102 * 739)
    
    results.append((fitted, estimate))
    print(estimate)

18.23648809997612
19.09034466289899
19.39166600334315
19.88454190877975
19.952307039189154
20.01525644087134
CPU times: user 6h 11min 32s, sys: 4min 44s, total: 6h 16min 16s
Wall time: 6h 17min 23s


## Задание № 4

Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:

<center>$Q_{june}=\frac{1}{R∗739∗6}\sum_{r=1}^R\sum_{T=2016.05.01-00:00}^{T=2016.05.31-23:00}\sum_{i=1}^6\mid{\widehat{y}}^r_{T|T+i}−{\widehat{y}}^r_{T+i}\mid.$</center>

Убедитесь, что ошибка полученных прогнозов, рассчитанная согласно функционалу Q, определённому на прошлой неделе, уменьшилась по сравнению с той, которую вы получили методом индивидуального применения моделей ARIMA. Если этого не произошло, попробуйте улучшить ваши модели.

В данном случае, нам остается просуммировать оценки, полученные ранее, и разделить на количество задач.

In [444]:
estimate = 0
for i in range(6):
    estimate += results[i][1]
print(estimate / 6)

19.428434025843085


Полученный результат значительно лучше прошлого (**32.10632**).

## Задание № 5

Итоговыми моделями постройте прогнозы для каждого конца истории от ***2016.05.31 23:00 до 2016.06.30 17:00*** и запишите все результаты в один файл в формате ***geoID, histEndDay, histEndHour, step, y***. Здесь ***geoID*** — идентификатор зоны, ***histEndDay*** — день конца истории в формате ***id***,***y***, где столбец ***id*** состоит из склеенных через подчёркивание идентификатора географической зоны, даты конца истории, часа конца истории и номера отсчёта, на который делается предсказание ***(1-6)***; столбец ***y*** — ваш прогноз.

Построение прогноза на контрольной выборке.

In [445]:
file_names =  ["1. first_prediction", "2. second_prediction", "3. third_prediction",
               "4. forth_prediction", "5. fifth_prediction", "6. sixth_prediction"]
for i in range(6):
    encoded_data = encoder.transform(result_data[i][2][cat_attributes].T.to_dict().values())
    cat_att = pd.DataFrame(encoded_data, columns=cat_columns)
    cat_att.index = result_data[i][2].index

    X_test = pd.concat([result_data[i][2][a], cat_att], axis=1)
    
    estimator = results[i][0]
    prediction = estimator.predict(X_test)
    res_shape = result_data[i][2]["region"].get_values().shape[0]
    
    prediction = pd.DataFrame(np.concatenate((result_data[i][2]["region"].get_values().reshape(res_shape, 1),
                                              abs(prediction.astype(int)).reshape(res_shape, 1)),
                                             axis=1),
                              columns=["region", "trips"])
    
    prediction.to_csv("new_result/" + file_names[i] + ".csv", index=False)

Подготовка массива дат для оформления результата.

In [446]:
from datetime import timedelta, datetime

def daterange(date1, date2):
    dates = []
    for n in range(int((date2 - date1).days * 24)+ 19):
        dates.append(date1 + timedelta(n / 24))
    return dates

start_dt = datetime(2016, 5, 31, 23)
end_dt = datetime(2016, 6, 30, 17)
dates = daterange(start_dt, end_dt)

date_list = list(map(lambda date: np.repeat(date.strftime("_%Y-%m-%d_") + str(date.hour) + "_", 102), dates))
date_list = np.concatenate(date_list)

In [454]:
file_path = np.sort(glob("new_result/*.csv", recursive=True))
solution_data = []
for i in range(len(file_path)):
    path = file_path[i]
    result = pd.read_csv(path)[:len(dates * 102)]
    solution_data.append(result)
    print(result.shape)

(72930, 2)
(72930, 2)
(72930, 2)
(72930, 2)
(72930, 2)
(72930, 2)


Сохранение в необходимом формате для **Kaggle** данные прогнозов.

In [456]:
solutions = []
for i in range(6):
    res = solution_data[i]
    regions = list(map(lambda reg: reg.split("_")[1], solution_data[i]["region"]))

    ids = []
    for j in range(len(date_list)):
        ids.append(regions[j] + date_list[j])
    
    res["id"] = list(map(lambda _id: _id + str(i+1), ids))
    res["y"] = res.trips
    solutions.append(res[["id", "y"]])

In [457]:
pd.concat(solutions).sort_values("id").to_csv("new_solution.csv", index=False)

## Задание № 6

Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

https://www.kaggle.com/submissions/6569525/6569525.zip